In [1]:
%cd ../src

/Users/adrian/Dropbox/college/4th Year/Modern Information Management/project/src


# Question Page Analysis

Below we have our exam page parser. It parses all the indexes of questions and builds up a model of what the exam paper looks like. It works by finding all the indexes in the paper. An index is the numerical, alphabetical or roman characters that we reference questions by (e.g. 1., (b), [c]). We then iterate over these indexes and assume each denotes a question. Each consectutive index of the same type (numerical, alphabetical or roman) describe a list of questions. If the index type changes, we assume that this index denotes a sub-question of the previous index. 

For example, say we the indices: `1., (a), (b), i. (c), (d), 2`

1. `1.` denotes the first question. Stack: `[]`
2. `(a)` is a different class than `1.` (alphabetical vs numerical) so `(a)` is a sub-question of `1.`. `[Question[1.]]`
3. `(b)` is the same class as `(a)` so it is also a sub-question of `1.`. `[Question[1.], Question[(a)]]`
4. `i.` is a different class than `(b)` so it's a sub-question of `(b)`. `[Question[1.], Question[(b)]]`
5. `(c)` is a different class than `i.` but the same class as `(b)` and the next value (`3`) so it's a sub-question of `1.`. `[Question[1.], Question[(b)], Question[i.]]`
6. `(d)` is the same class as `(c)` so it's a sub-question of `1.`. `[Question[1.], Question[(c)]]`
7. `2.` is a different class than `(d)` so it's a top-level question, alongside `1.`. `[Question[1.]]`
8. `[Question[1.], Question[2.]]`

The final structure would look like this:

    1.
    |-- (a)
      | (b)
      |  |-- i.
      | (c)
      | (d)
    2.
    
### Considerations
* Sometimes questions have marks in the form of an index e.g. `(10)`
 
  To combat this, we look at the previous index `i` of the same type (i.e. numerical) and see if it's value is `i - 1`. 
  
  In future, we could refine it to look to see if two indexes are sitting right i.e. marks at the end of a question and the next question's index. If this case happens we could drop the marks. We have to be careful of dropping a multiple choice answer that could have been an image (i.e. no text, just an index) from being parsed as marks in that situation (See chemistry paper extraction).
  
  
* Sometimes sentences end with a single letter or digit e.g. `potassium K.`

  To account for this, we will parse the letter or digit as an index and then look at it's position relative to the previous index. If for instance, the index was alphabetical with the value "K" which translates to position 11 in a list, if the last index is not the alphabetical index with position 10, we can safely discard the "K". Unfortunately, this does come with caveat that indices *could* line up.
    
### Caveats
Currently this algorithm has some caveats:

* We cannot determine between multiple choice answers and sub-questions.
* We cannot determine between a sentence ending in `<SPC>[0-9].` and an index.
* We cannot determine inline references to indices.

These caveats are all caused by sampling on the body of text. They could be alleviated if we moved to a more detailed library such as PDFMiner and extracted geometries associated with different bodies of text. It would solve most if not all the problems of ambiguity.

So, let's build the parser.

In [2]:
# %load paper.py
import slate
import logging
import pyparsing as pp

from index import Index
from question import Question
from section import Section

class Paper(Question):
    # Let's define our parser
    __ = pp.White().suppress()
    _ = pp.White(exact=1).suppress() | pp.LineEnd() | pp.LineStart()
    ___ = pp.Optional(__)

    # Define tokens for numerical, alpha and roman indices
    # Max two digits for numerical indices because lecturers aren't psychopaths
    index_digit = pp.Word(pp.nums, max=2).setParseAction(lambda s, l, t: [Index("decimal", int(t[0]))])("[0-9]") 
    index_alpha = pp.Word(pp.alphas, exact=1).setParseAction(lambda s, l, t: [Index("alpha", t[0])])("[a-z]")
    index_roman = pp.Word("ivx").setParseAction(lambda s, l, t: [Index("roman", t[0])])("[ivx]") # We only support 1-100 roman numerals
    index_type = (index_digit | index_roman | index_alpha)("index")

    # Define token for ("Question" / "Q") + "."
    question = (pp.CaselessLiteral("Question") + pp.Optional("."))("question")

    # Define tokens for formatted indices e.g [a], (1), ii. etc.
    index_dotted = index_type + pp.Literal(".").suppress()
    index_round_brackets = pp.Literal("(").suppress() + index_type + pp.Literal(")").suppress()
    index_square_brackets = pp.Literal("[").suppress() + index_type + pp.Literal("]").suppress()
    index_question = pp.Word("qQ", exact=1).suppress() + pp.Optional(".").suppress() + index_type + pp.Optional(".").suppress()

    # Define final index token with optional Question token before formatted index
    index = (
        # Whitespace is required before each index 
        _ + \
        # Optional "Question." before
        pp.Optional(question + ___).suppress() + \
        # The index
        (index_question | index_dotted | index_round_brackets | index_square_brackets) + \
        # Required whitespace *after* index
        _
    )

    # Define a section header
    section = (pp.CaselessLiteral("Section").suppress() + __ + index_type + _).setParseAction(lambda s, l, t: [t[0].setSection(True)]).setName("section")

    # Entry point for the parser
    entry = section ^ index

    def __init__(self, document=None):
        """The Paper class describes a Exam paper.
        
        Here we parse questions and store them in a neat array.
        """
        self.questions = []

        if document != None:
            self.parse(document)

    def parse(self, document):
        """Parse the paper's questions.
        
        This function takes in the parsed PDF document, which is an 
        list of strings, one for each page. We assume the document
        follows the Paper Specification in notebooks/Exam Paper Feature
        Exraction.ipynb.

        Args:
            document (List[str]): The list of pages.
        """
        # Parse the front page
        self.parseFrontPage(document[0])

        # Parse the rest of the pages
        self.parsePages(document[1:])

    def parseFrontPage(self, front_page):
        """Parse the front page of a paper.

        Args:
            front_page (str): The front page of a paper.
        """

        logging.info("Parsing front page of \"%s\".." % front_page[:30])

    def parsePages(self, pages):
        """Parse a page in a paper.

        We have a pretty complicated sorting algorithm

        Args:
            page (str): A page within the paper.
        """

        if isinstance(pages, list):
            pages = ' '.join([page for page in pages])


        stack = [self]

        for token, u, v in Paper.entry.scanString(pages):
            index = token[0]
            last = stack[-1]
            
            # If we have a section, 
            # we just push the section onto the stack so the
            # following questions are pushed into it.
            if index.is_section:
                section = Section(index)
                last.pushQuestion(section)
                stack.append(section)
            else:
                new_question = Question(index)

                # If we have a paper at the top of the stack, it means
                # we are operating on a top level question. We don't need
                # to compare the index to the last index because there is
                # none. In this case, we just push the question onto the
                # paper.
                if isinstance(last, Paper):
                    last.pushQuestion(new_question)
                    stack.append(new_question)
                # Now we compare the index types, if they're equal
                # We pop the last from the stack, push the question
                # into the new head of the stack and then push the
                # question itself onto the stack.
                elif index.index_type == last.index.index_type and last.index.isNext(index):
                    stack.pop()
                    last = stack[-1]
                    last.pushQuestion(new_question)
                    stack.append(new_question)
                # If they're not equal, it means they're starting a new list. This new list
                # is now a bunch of sub questions for the head of the stack
                elif index.index_type != last.index.index_type and last.index.value == 0:
                    last.pushQuestion(new_question)
                    stack.append(new_question)
                # Okay so they're none of the above, now we check if the stack[-2] index
                # is the of the same type. If it is, it means were moving out of the
                # sub list and back up one in the stack. We need to pop the head from the
                # stack and *then* append it to the new stack[-2] and push it onto the stack
                elif index.index_type != last.index.index_type:
                    parent = stack[-2]
                    
                    if parent.index and index.index_type == parent.index.index_type:
                        stack.pop()
                        stack.pop()
                        last = stack[-2]
                        last.pushQuestion(new_question)
                        stack.append(new_question)

    @staticmethod
    def from_pdf(path):
        """Parse a paper from a PDF.

        Args:
            path (str): The path to the PDF.

        Returns:
            A new Paper object from the PDF.
        """

        with open(path) as pdf:
            return Paper(slate.PDF(pdf))

Parser built, let's try it on some sample papers.

In [3]:
from os import listdir
from os.path import isfile, join
from IPython.display import HTML
import codecs
import slate

DATA_DIR = "../data"

# Grab the files
exams = [pdf for pdf in listdir(DATA_DIR) if isfile(join(DATA_DIR, pdf))]
output = ""

# Let's parse the exam papers
for i in range(len(exams)):
    with open(join(DATA_DIR, exams[i])) as paper:
        output += "<h3>Page 1 of %s </h3><p>" % exams[i]
        
        pages = exams[i] = slate.PDF(paper)
        
        pages = ' '.join([page for page in pages[1:]])

        skew = 0
        for i, u, v in Paper.entry.leaveWhitespace().scanString(pages, overlap=True):
            t = pages[u+skew:v+skew]
            b = "<b style='color:red'>%s</b>"
            pages = pages[:u+skew] + (b % t) + pages[v+skew:]
            skew += len(b) - 2

        output += pages.decode("latin-1")
        
HTML(output)

### Building the question tree
Alright, so things looking mildly alright there. Still a couple of errors but nothing a little duck tape can't fix. Let's try actually building the pages with `parsePages`.

In [4]:
# Grab an example paper
example = exams[0]

Paper(document=example)

AttributeError: 'And' object has no attribute 'index_type'